# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from google_key import api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("Output/weather_csv.csv")
weather_df.describe()

,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,18.643385,19.303316,68.200990,67.329861,51.196181,8.133403
std,32.824753,91.011749,15.337732,20.045137,36.372489,5.615715
min,-54.800000,-179.170000,31.440000,10.000000,0.000000,0.250000
25%,-7.915000,-61.267500,57.000000,54.000000,19.750000,3.800000
50%,21.090000,24.700000,69.800000,70.000000,51.000000,6.930000
75%,45.380000,100.342500,80.160000,83.000000,87.000000,11.410000
max,78.220000,177.480000,107.460000,100.000000,100.000000,36.910000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]

In [4]:
# Configure gmaps
gmaps.configure(api_key=api_key)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                     max_intensity=max(humidity),
                                     dissipating=True,
                                    point_radius = 15)


# Add layer
fig.add_layer(humidity_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
better_conditions_df = weather_df[(weather_df["Max Temperature (F)"]>=70) & 
                                  (weather_df["Max Temperature (F)"]<=80) &
                                 (weather_df["Wind Speed"]<=10) &
                                 (weather_df["Cloudiness"]==0)]
better_conditions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 228 to 521
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 7 non-null      object 
 1   Lat                  7 non-null      float64
 2   Lng                  7 non-null      float64
 3   Max Temperature (F)  7 non-null      float64
 4   Humidity             7 non-null      int64  
 5   Cloudiness           7 non-null      int64  
 6   Wind Speed           7 non-null      float64
 7   Country              7 non-null      object 
dtypes: float64(4), int64(2), object(2)
memory usage: 504.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# geocoordinates
target_coordinates = better_conditions_df[["Lat", "Lng"]]
target_coordinates = target_coordinates.values.tolist()
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

hotel_name=[]

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for coordinates in target_coordinates:
    coord = f"{coordinates[0]},{coordinates[1]}"
    # set up a parameters dictionary
    params = {
        "location": coord,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": api_key}
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name.append(response["results"][0]["name"])
    except:
        hotel_name.append("Hotel Not Found")

32.69,128.84
-13.2,49.05
41.18,29.61
-9.35,28.73
45.97,42.7
-8.42,20.74
31.61,34.76


In [44]:
hotel_df=better_conditions_df
hotel_df["Hotel Name"] = hotel_name
hotel_df.head()

/Users/tiagolopes/opt/anaconda3/envs/LearnPython/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Hotel Name
228,fukue,32.69,128.84,71.04,74,0,6.11,JP,SERENDIP HOTEL GOTO
236,ambilobe,-13.20,49.05,70.29,69,0,4.90,MG,Hotel National
292,sile,41.18,29.61,73.00,66,0,1.01,TR,The Sign Sile Hotel & Spa
421,nchelenge,-9.35,28.73,71.24,60,0,2.01,ZM,CM Guest house
459,bolshaya dzhalga,45.97,42.70,70.47,51,0,6.53,RU,Hotel Not Found


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))